# voca quiz mid high 검수
- 문제를 쉽게 풀려고 해라
- 쉽게쉽게쉽게쉽게쉽게 생각...

In [1]:
import pandas as pd
import re

In [2]:
path = "C:\\Users\\WEAVERSMIND\\Desktop\\Weaversmind\\Word\\PreviewQuiz_Word\\MidHigh\\Quiz_Voca_Mid(total).csv"
df = pd.read_csv(path)
df.head()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa6 in position 14: invalid start byte

In [ ]:
sample_df = df[:5]
sample_df.head(1)
data = sample_df

In [ ]:
def check_voca_quiz(data:pd.DataFrame) -> pd.DataFrame:
    
    error_state = False

    # 1. korean_translation 확인
    check_symbol_pattern = r"\[\[.*?\]\]"
    blank = r"_+"

    for i in range(len(data)):
        # 1-1 [[]]가 있는지 확인
        if not re.search(check_symbol_pattern, data['korean_translation'][i]):
            error_state = True
            print(f"korean translation {i} 번째 행 {data['answer'][i]} 에 [[]] 오류가 있음")
        
        # 1-2 영어가 있는지 확인
        cleaned_text = re.sub("TV", "",data['korean_translation'][i])
        if re.search(r"[a-zA-Z]", cleaned_text):
            error_state = True
            print(f"korean translation {i} 번째 행 {data['answer'][i]} 에 영어가 있음")

        # 1-3 밑줄이있는지 확인
        underscores = re.findall(blank, data['fill_in_blank_quiz'][i])
        if len(underscores) < 1:
            error_state = True
            print(f"fill in blank quiz {i} 번째 행 {data['answer'][i]} 에 __ 밑줄이 없음")
    
    if error_state==False:
        return f"발견된 에러가 없음!!"

- 다음 부터는 그냥 데이터를 만들자 뭐가 오류인지
    - 이렇게 하니씩 확인하려고 하니 힘듬..

In [ ]:
check_voca_quiz(df)

korean translation 1230 번째 행 height 에 영어가 있음
korean translation 1545 번째 행 share 에 영어가 있음
korean translation 2371 번째 행 worth 에 [[]] 오류가 있음
korean translation 3415 번째 행 campfire 에 [[]] 오류가 있음
korean translation 4297 번째 행 both A and B 에 영어가 있음
korean translation 5215 번째 행 mix A with B 에 영어가 있음
korean translation 5510 번째 행 part 에 [[]] 오류가 있음
korean translation 5839 번째 행 read 에 영어가 있음
korean translation 6176 번째 행 skirt 에 [[]] 오류가 있음


In [ ]:
check_symbol_pattern = r"\[\[.*?\]\]"
blank = r"_+"

for i in range(len(data)):
    if not re.search(check_symbol_pattern, data['korean_translation'][i]):
        print(f"korean translation {i} 번째 행에 [[]] 오류가 있음")

    if re.search(r"[a-zA-Z]", data['korean_translation'][i]):
        print(f"korean translation {i} 번째 행에 영어가 있음")

    underscores = re.findall(blank, data['fill_in_blank_quiz'][i])
    if len(underscores) < 1:
        print(f"fill in blank quiz {i} 번째 행에 __ 밑줄이 없음")
    
    

korean translation 2 번째 행에 영어가 있음


In [ ]:
data['korean_translation']

0    그녀는 독특한 [[강세]]로 말해요.
1       그녀는 [[억양]]으로 말해요.
2       그들은 TV에 [[광고]]해요.
3         회사에서는 [[광고]]해요.
4      아빠는 [[항공사]]에서 일해요.
Name: korean_translation, dtype: object

In [ ]:
data['fill_in_blank_quiz']

0           She speaks with a unique ___.
1                    She speaks with ___.
2                         They ___ on TV.
3    The company ___ their product today.
4                   My dad works for ___.
Name: fill_in_blank_quiz, dtype: object

# 영어 문장과 한글 문장의 뜻이 일치하는지 유효성 검사

In [64]:
from openai import OpenAI
from getpass import getpass
from jinja2 import Template
from openai.lib._parsing import type_to_response_format_param
from pydantic import BaseModel
import json
import pandas as pd
from tqdm import tqdm

In [65]:
api_key = getpass("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

In [66]:
sample = pd.read_csv('sample_sentence.csv', skiprows=2)
sample.columns.tolist()

['구분',
 'voca NO',
 'grade ID',
 'grade NO',
 'WordText',
 'WordMeaning',
 'PoS',
 '품사',
 'image ID',
 'WordID(SoundID)',
 'WordSymbol',
 'example_english',
 'example_korean',
 'new_example_english',
 'new_example_korean',
 'fill_in_blank_quiz',
 'korean_translation',
 'answer',
 'hint_eng',
 'hint_kor',
 '중고등1200 포함여부',
 '예시 단어 포함 여부',
 'answer/wordtext 일치여부',
 '워드P 매핑용',
 '워드P 매핑용 쿼리']

In [ ]:
prompt_template = Template(
"""
[PERSONA]
You are a translation review expert.

[POLICY]
1. Check if "example_sentence" and "example_sentence_kor" have the same meaning.
2. Classify the comparison result into one of the following states: "correct", "supplement", or "incorrect".
3. Provide a reason for the assigned state in the "reason" field.
4. The example sentence must align with 'WordText', 'WordMeaning', and 'PoSText'. If the meaning is different, the classification should be "incorrect".




[EXAMPLE 1 : correct]
```
Input:
    {
    "WordText" : "disappoint",
    "WordMeaning" : "실망시키다",
    "PoSText" : "동사",
    "example_sentence" : "I do not want to disappoint my friend."
    "example_sentence_kor" : "내 친구를 실망시키고 싶지 않아요."
    }

Output:
    {
    "WordText" : "disappoint",
    "state" : "correct",
    "reason": "The English and Korean sentences have the same meaning. 'I do not want to disappoint my friend.' is correctly translated as '내 친구를 실망시키고 싶지 않아요.' without any significant differences in nuance or intent."
    }
```       

[EXAMPLE 2 : supplement]
```
Input:
    {
    "WordText" : "favor",
    "WordMeaning" : "좋아하다",
    "PoSText" : "동사",
    "example_sentence" : "I favor my teacher because she is kind."
    "example_sentence_kor" : "저는 선생님을 좋아해요.",
    }

Output:
    {
    "WordText" : "favor",
    "state" : "supplement",
    "reason": "The Korean sentence '저는 선생님을 좋아해요.' means 'I like my teacher,' which conveys a similar idea but lacks the reason provided in the English sentence ('because she is kind'). The nuance of 'favor' implying preference is also not fully captured."
    }

```

[EXAMPLE 3 : incorrect]
```
Input:
    {
    "WordText" : "alarm",
    "WordMeaning" : "놀람, 경보",
    "PoSText" : "명사",
    "example_sentence" : "My alarm rings loudly to wake me up."
    "example_sentence_kor" : "내 알람이 크게 울려서 나를 깨워요."
    }

Output:
    {
    "WordText" : "alarm",
    "state" : "incorrect",
    "reason": "The WordMeaning '놀람, 경보' refers to 'alarm' as a noun meaning 'surprise' or 'warning,' but in the example sentence, 'alarm' refers to an 'alarm clock' (알람 시계). This mismatch in meaning makes the translation incorrect."
    }
```

[Input]
```

WordText : {{WordText}}
WordMeaning : {{WordMeaning}}
PoSText : {{PoSText}}
example_sentence : {{example_sentence}}
example_sentence_kor : {{example_sentence_kor}}
```                   
"""     
)

In [68]:
class ExampleSentence(BaseModel):
    WordText : str
    state : str
    reason : str

In [69]:
response_format = type_to_response_format_param(ExampleSentence)

In [70]:
prompt = prompt_template.render(
    WordText = "alarm",
    WordMeaning = "알람,자명종",
    PoSText = "동사",
    example_sentence = "My alarm rings loudly to wake me up.",
    example_sentence_kor = "내 경보가 크게 울려서 나를 깨워요.",
) 

In [71]:
def completion(prompt : str) -> str:
    response = client.beta.chat.completions.parse(
        model = 'o3-mini',
        reasoning_effort='low',
        messages = [
            {"role" : "system", "content" : "You will provide a fill-in-the-blank quiz for each given word, a Korean translation, and a hint."},
            {"role" : "user", "content" : prompt}
        ],
        response_format = ExampleSentence,
    )
    return response.choices[0].message.parsed

In [72]:
print(prompt)


[PERSONA]
You are a translation review expert.

[POLICY]
1. Check if "example_sentence" and "example_sentence_kor" have the same meaning.
2. Classify the comparison result into one of the following states: "correct", "supplement", or "incorrect".
3. Provide a reason for the assigned state in the "reason" field.
4. The example sentence must align with 'WordText', 'WordMeaning', and 'PoSText'. If the meaning is different, the classification should be "incorrect".




[EXAMPLE 1 : correct]
```
Input:
    {
    "WordText" : "disappoint",
    "WordMeaning" : "실망시키다",
    "PoSText" : "동사",
    "example_sentence" : "I do not want to disappoint my friend."
    "example_sentence_kor" : "내 친구를 실망시키고 싶지 않아요."
    }

Output:
    {
    "WordText" : "disappoint",
    "state" : "correct",
    "reason": "The English and Korean sentences have the same meaning. 'I do not want to disappoint my friend.' is correctly translated as '내 친구를 실망시키고 싶지 않아요.' without any significant differences in nuance or inte

In [73]:
response = completion(prompt)

In [74]:
response_output = json.dumps(response.dict(), ensure_ascii=False, indent = 4)
print(response_output)

{
    "WordText": "alarm",
    "state": "incorrect",
    "reason": "The WordMeaning provided includes both '알람' and '자명종', but the PoSText is '동사', suggesting a verbal usage. The English example sentence 'My alarm rings loudly to wake me up.' implies a noun usage (likely referring to an alarm clock), and the Korean translation '내 경보가 크게 울려서 나를 깨워요.' uses '경보', meaning a warning system rather than an alarm clock. This mismatch in both the part of speech and the meaning of the translated word makes the translation incorrect."
}


In [75]:
def ExampleValid(api_key:str, prompt_template:str, response_format, data:pd.DataFrame):
    # api 호출하기
    client = OpenAI(api_key=api_key)

    # 결과 저장
    results = []

    # 검수 prompt 생성
    for i in tqdm(range(len(data)), total = len(data)):
        prompt = prompt_template.render(
            WordText = data.loc[i,"WordText"],
            WordMeaning = data.loc[i,"WordMeaning"],
            PoSText = data.loc[i,"품사"],
            example_sentence = data.loc[i,'new_example_english'],
            example_sentence_kor = data.loc[i, 'new_example_korean']
        )

        response = client.beta.chat.completions.parse(
            model = 'o3-mini',
            reasoning_effort = "low",
            messages = [
                {"role" : "system", "content" : "Check if the given English sentence (`example_sentence`) and its Korean translation (`example_sentence_kor`) have the same meaning." },
                {"role" : "user", "content" : prompt}
            ],
            response_format = response_format
        )
        
        # 응답 결과 저장
        output = response.choices[0].message.parsed
        response_output = json.dumps(output.dict(), ensure_ascii=False, indent=2)
        results.append(response_output)

    return results

In [76]:
sample = sample[:100]

In [77]:
sampe_output = ExampleValid(api_key=api_key, 
             prompt_template=prompt_template,
             response_format=ExampleSentence,
             data=sample,
             )

100%|██████████| 100/100 [06:46<00:00,  4.07s/it]


In [78]:
print(sampe_output)

['{\n  "WordText": "accent",\n  "state": "correct",\n  "reason": "The English sentence \'My friend speaks English with a clear accent.\' and the Korean translation \'내 친구는 영어를 명확한 강세로 말해요.\' share the same meaning. The term \'accent\' as \'강세\' (the emphasis on syllables) is used accurately in both, and the overall context is maintained without any loss of meaning or nuance."\n}', '{\n  "WordText": "accent",\n  "state": "correct",\n  "reason": "The English sentence \'She speaks fluent English with a clear accent.\' is accurately translated as \'그녀는 뚜렷한 억양으로 영어를 말해요.\' Both sentences convey that her English is fluent and has a distinct accent, aligning with the word meaning \'억양\' and the part of speech \'명사\'."\n}', '{\n  "WordText": "advertise",\n  "state": "correct",\n  "reason": "The English sentence \'The big store advertises new toys on TV.\' and its Korean translation \'큰 가게가 새로운 장난감을 TV에서 광고해요.\' convey the same meaning. The translation accurately reflects the meaning of \'adver

In [79]:
import json

# Convert all string elements in list `a` into valid JSON objects
parsed_data = [json.loads(item) for item in sampe_output]

# Pretty-print the JSON list
sample_list = json.dumps(parsed_data, ensure_ascii=False, indent=2)
#print(sample_list)


In [80]:
from collections import Counter

check_list = []

for sample in eval(sample_list):
    count = sample['state']
    check_list.append(count)

state_count = Counter(check_list)
print(state_count)


Counter({'correct': 89, 'supplement': 9, 'incorrect': 2})


## jsonl 파일로 저장 준비 -> batch api 보내야함

In [82]:
def ExampleValid(data:pd.DataFrame, output_filename, response_format=response_format):
    # jsonl_data 저장
    jsonl_data = []

    # 검수 prompt 생성
    for i in range(len(data)):
        prompt = prompt_template.render(
            WordText = data.loc[i,"WordText"],
            WordMeaning = data.loc[i,"WordMeaning"],
            PoSText = data.loc[i,"품사"],
            example_sentence = data.loc[i,'new_example_english'],
            example_sentence_kor = data.loc[i, 'new_example_korean']
        )

        quiz_request = {
            "custom_id" : f"request-{i+1}",
            "method" : "POST",
            "url" : "/v1/chat/completions",
            "body" : {
                "model" : "o3-mini-2025-01-31",
                "messages" : [
                    {"role": "system", "content": "Check if the given English sentence (`example_sentence`) and its Korean translation (`example_sentence_kor`) have the same meaning."},
                    {"role": "user", "content": prompt}
                ],
                "response_format" : response_format
            }
        }

        jsonl_data.append(quiz_request)
        
        # 파일 쓰기
        
        with open(output_filename, 'w', encoding="utf-8") as jsonl_file:
            for item in jsonl_data:
                jsonl_file.write(json.dumps(item, ensure_ascii=False) + '\n')

    print(f"JSONL 파일 생성 완료 : {output_filename}-{i+1}")

In [85]:
df = pd.read_csv("sample_sentence.csv", skiprows=2)
print(len(df))
df1 = df[:1000]
print(len(df1))
df2 = df[1000:2000]
print(len(df2))
df3 = df[2000:3000]
print(len(df3))
df4 = df[3000:4000]
print(len(df4))
df5 = df[4000:5000]
print(len(df5))
df6 = df[5000:6000]
print(len(df6))
df7 = df[6000:]
print(len(df7))

print(f'total : {len(df1) + len(df2) + len(df3) + len(df4) + len(df5) + len(df6) + len(df7)}')

#인덱스 리셋
df2 = df2.reset_index()
df3 = df3.reset_index()
df4 = df4.reset_index()
df5 = df5.reset_index()
df6 = df6.reset_index()
df7 = df7.reset_index()

7251
1000
1000
1000
1000
1000
1000
1251
total : 7251


In [86]:
ExampleValid(df1,"VocaQuiz_MidHigh_batch_1.jsonl")
# ExampleValid(df2,"VocaQuiz_MidHigh_batch_2.jsonl")
# ExampleValid(df3,"VocaQuiz_MidHigh_batch_3.jsonl")
# ExampleValid(df4,"VocaQuiz_MidHigh_batch_4.jsonl")
# ExampleValid(df5,"VocaQuiz_MidHigh_batch_5.jsonl")
# ExampleValid(df6,"VocaQuiz_MidHigh_batch_6.jsonl")
# ExampleValid(df7,"VocaQuiz_MidHigh_batch_7.jsonl")

JSONL 파일 생성 완료 : VocaQuiz_MidHigh_batch_1.jsonl-1000


# batch api로 보내기